In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import shap
import lime
from lime.lime_tabular import LimeTabularExplainer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import mean_squared_error
shap.initjs()
import re
import nltk
from wordcloud import WordCloud
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'xgboost'

In [4]:
drugs = pd.read_csv("drugsComTrain_raw.csv")
drugs_test=pd.read_csv("drugsComTest_raw.csv")

In [5]:
drugs.head()
drugs_test.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10,28-Feb-12,22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8,17-May-09,17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9,29-Sep-17,3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9,5-Mar-17,35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9,22-Oct-15,4


In [6]:
len(drugs)
len(drugs_test)

53766

In [64]:
drugs[drugs['drugName'].str.contains('/')]

,uniqueID,drugName,condition,review,rating,date,usefulCount
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1
45,111409,Ledipasvir / sofosbuvir,Hepatitis C,"""At initial testing my VL was over 6 million. ...",10,22-Sep-15,0
46,111474,Ledipasvir / sofosbuvir,Hepatitis C,"""Side effects are light- fatigue and a bit of ...",9,31-Dec-14,94
61,192806,Drospirenone / ethinyl estradiol,Birth Control,"""I was put on Yasmin for 6 months to regulate ...",3,11-Apr-11,8
...,...,...,...,...,...,...,...
161224,86508,Magnesium sulfate / potassium sulfate / sodium...,Bowel Preparation,"""Prepping for my first colonoscopy. Took the f...",7,13-Feb-14,14
161232,49137,Ethinyl estradiol / levonorgestrel,Birth Control,"""I&#039;ve been on aleese a month now and it&#...",3,1-Aug-16,4
161233,104148,Ethinyl estradiol / levonorgestrel,Birth Control,"""This birth control is the best! I got it beca...",9,22-Jun-13,5
161269,73058,Ethinyl estradiol / norethindrone,Birth Control,"""When I first starting taking Lo Loestrin Fe, ...",10,20-Apr-15,13


In [65]:
print(6586/53766)

0.12249376929658148


In [66]:
#Valori unici per drugName

drugs['drugName'].nunique()

3436

In [67]:
drugs[drugs['drugName'].str.contains('/')].nunique()

uniqueID       19870
drugName         327
condition        168
review         18932
rating            10
date            3393
usefulCount      171
dtype: int64

In [68]:
drugs[drugs['drugName'].str.contains('/')].info()

<class 'pandas.core.frame.DataFrame'>
Index: 19870 entries, 4 to 161291
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   uniqueID     19870 non-null  int64 
 1   drugName     19870 non-null  object
 2   condition    19454 non-null  object
 3   review       19870 non-null  object
 4   rating       19870 non-null  int64 
 5   date         19870 non-null  object
 6   usefulCount  19870 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 1.2+ MB


In [69]:
# Uniamo farmaco e condizione
drugs['farmaco_condizione'] = drugs['drugName'].astype(str) + " " + drugs['condition'].astype(str)

In [70]:
drugs.head(10)

,uniqueID,drugName,condition,review,rating,date,usefulCount,farmaco_condizione
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,Valsartan Left Ventricular Dysfunction
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,Guanfacine ADHD
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,Lybrel Birth Control
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8,3-Nov-15,10,Ortho Evra Birth Control
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9,27-Nov-16,37,Buprenorphine / naloxone Opiate Dependence
5,155963,Cialis,Benign Prostatic Hyperplasia,"""2nd day on 5mg started to work with rock hard...",2,28-Nov-15,43,Cialis Benign Prostatic Hyperplasia
6,165907,Levonorgestrel,Emergency Contraception,"""He pulled out, but he cummed a bit in me. I t...",1,7-Mar-17,5,Levonorgestrel Emergency Contraception
7,102654,Aripiprazole,Bipolar Disorde,"""Abilify changed my life. There is hope. I was...",10,14-Mar-15,32,Aripiprazole Bipolar Disorde
8,74811,Keppra,Epilepsy,""" I Ve had nothing but problems with the Kepp...",1,9-Aug-16,11,Keppra Epilepsy
9,48928,Ethinyl estradiol / levonorgestrel,Birth Control,"""I had been on the pill for many years. When m...",8,8-Dec-16,1,Ethinyl estradiol / levonorgestrel Birth Control


In [71]:
# Uniamo farmaco e condizione  nella review
drugs['review'] = drugs['farmaco_condizione'] + " " + drugs['review'].astype(str) + " " + drugs['farmaco_condizione']

In [72]:
print(drugs["review"][0])

Valsartan Left Ventricular Dysfunction "It has no side effect, I take it in combination of Bystolic 5 Mg and Fish Oil" Valsartan Left Ventricular Dysfunction


In [73]:
# Statistiche descrittive generali per il dataframe drugs
print("Statistiche descrittive (numeriche):")
print(drugs.describe())

print("\nStatistiche descrittive (categoriche):")
print(drugs.describe(include=['object']))

print("\nValori nulli per colonna:")
print(drugs.isnull().sum())

Statistiche descrittive (numeriche):
            uniqueID         rating    usefulCount
count  161297.000000  161297.000000  161297.000000
mean   115923.585305       6.994377      28.004755
std     67004.445170       3.272329      36.403742
min         2.000000       1.000000       0.000000
25%     58063.000000       5.000000       6.000000
50%    115744.000000       8.000000      16.000000
75%    173776.000000      10.000000      36.000000
max    232291.000000      10.000000    1291.000000

Statistiche descrittive (categoriche):
              drugName      condition  \
count           161297         160398   
unique            3436            884   
top     Levonorgestrel  Birth Control   
freq              3657          28788   

                                                   review      date  \
count                                              161297    161297   
unique                                             161247      3579   
top     Tretinoin Acne "I have posted early b

In [74]:
drugs[drugs['review'].str.contains('Tretinoin Acne "I have posted')]

,uniqueID,drugName,condition,review,rating,date,usefulCount,farmaco_condizione
121185,24046,Tretinoin,Acne,"Tretinoin Acne ""I have posted early before abo...",9,5-Aug-16,2,Tretinoin Acne
123669,24045,Tretinoin,Acne,"Tretinoin Acne ""I have posted early before abo...",9,5-Aug-16,5,Tretinoin Acne
145348,24044,Tretinoin,Acne,"Tretinoin Acne ""I have posted early before abo...",9,6-Aug-16,31,Tretinoin Acne


In [75]:
# Stampo tre review a scelta dagli indici 0, 1 e 2
print(drugs['review'].iloc[121185])
print()
print(drugs['review'].iloc[123669])
print()
print(drugs['review'].iloc[145348])
print()

Tretinoin Acne "I have posted early before about my 1st &amp; 2nd week using Retin A 0.025 cream. My 3rd week was much better than my second week, some of my acne stared to calm down and my face felt less bumpy. During this week I started to use the cream 4 times a week because my cheeks were hurting. My 4th week was the most improvement. I started to use the cream every night again since my face had stop hurting. During the 4th week most of my pimples disappeared and my skin felt smoother, I still had some pimples popping up but not as much. Now my cheeks are pretty smooth with a couple pimples but my cheeks are really red from the scars that were left behind from the pimples. I&#039;m hoping they will slowly fade later. Will keep updating!" Tretinoin Acne

Tretinoin Acne "I have posted early before about my 1st &amp; 2nd week using Retin A 0.025 cream. My 3rd week was much better than my second week, some of my acne stared to calm down and my face felt less bumpy. During this week I 

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))  # oppure 'italian'

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # rimuove punteggiatura
    text = re.sub(r'\d+', '', text)      # rimuove numeri
    text = re.sub(r"s+[a-zA-Z]s+", ' ', text) #rimuove singole lettere isolate
    text = re.sub(r's+', ' ', text) #rimuove doppi spazi


    words = text.split()
    words = [w for w in words if w not in stop_words and len(w) > 2]
    return " ".join(words)